In [ ]:
import tpot2
import sklearn.metrics
import sklearn
import sys
import pickle as pkl
import pandas as pd
import os

# display stuff
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# variables used throughout
task_id_lists = [167104, 167184, 167168, 167161, 167185, 189905, 167152, 167181, 189906, 189862, 167149, 189865, 167190, 189861, 189872, 168794, 189871, 168796, 168797, 75097, 126026, 189909, 126029, 126025, 75105, 168793, 189874, 167201, 189908, 189860, 168792, 167083, 167200, 168798, 189873, 189866, 75127, 75193]
data_dir = '../../Results_Prelim/'
replicates = 30
exp_dirs = ['Base/', '10/Lexicase/','30/Lexicase/','50/Lexicase/']
seeds = [0,1200,2400,3200]
proportions = [0.0, 0.1, 0.3, 0.5]
data_pkl = 'data.pkl'
scores_pkl = 'scores.pkl'
failed_pkl = 'failed.pkl'
scores_key = ['train_auroc','train_accuracy','train_balanced_accuracy','train_logloss','auroc','accuracy','balanced_accuracy','logloss','taskid','selection','seed']

In [ ]:
# data holders
SCORES_DF = []
OVER_TIME_DF = []

for cur_dir,seed, proportion in zip(exp_dirs,seeds, proportions):

    dir = data_dir + cur_dir

    if proportion == 0.0:
        acro = 'base'
    elif proportion == 0.1:
        acro = 'lex-10%'
    elif proportion == 0.3:
        acro = 'lex-30%'
    elif proportion == 0.5:
        acro = 'lex-50%'

    print('dir:',dir)
    print('acro:',acro)

    for subdir, dirs, files in os.walk(dir):
        # skip root dir
        if subdir == dir:
            continue
        # skip failed (will fix on the go)
        if os.path.exists(subdir + failed_pkl):
            continue

        # scores dictionary data (swap wit upper block if we care about seed, then pass seed down)
        scores_dict = pkl.load(open(subdir + scores_pkl,'rb'))
        scores_df = pd.DataFrame([scores_dict],  columns=scores_dict.keys())[scores_key]
        scores_df['proportion'] = float(int(proportion[:-1]) / 100.0)
        scores_df['acro'] = acro
        SCORES_DF.append(scores_df)

        # over time data
        data_df = pkl.load(open(subdir + data_pkl,'rb'))
        data_df['taskid'] = scores_dict['taskid']
        data_df['proportion'] = proportion
        data_df['acro'] = acro
        OVER_TIME_DF.append(data_df)


final_data = pd.concat(OVER_TIME_DF)
final_scores = pd.concat(SCORES_DF)

In [ ]:
final_data.to_csv(path_or_buf='ot_data.csv', index=False)
final_scores.to_csv(path_or_buf='scores.csv', index=False)